In [2]:
import os
import ROOT as r
dict={}
trees=["NOMINAL"]
with open("../../sys_trees.txt","r") as file:
    line=file.readline()
    while line:
        trees.append(line.rstrip('\n'))
        line=file.readline()

samples=["Signal_Sherpa","Signal_PoPy","Zjets","VV"]


In [2]:


# FILLING DICTIONARY WITH SYS-RELATED TREES 

### the dictionary will have the following structure "systematic_name":[fake1p,truth1p,fake3p,truth3p]

for sam in samples:
    for sys in trees[1:]:
        file=r.TFile.Open(sam+sys+".root")
        prong1=file.Get("tau_matched_1p_dphi_bdte_btag_iso_rnn_pte_omega_mle_mreco_tpt")
        prong3=file.Get("tau_matched_3p_dphi_bdte_btag_iso_rnn_pte_omega_mle_mreco_tpt")
        dict[sam+sys]=[prong1.GetBinContent(1),prong1.GetBinContent(2),prong3.GetBinContent(1),prong3.GetBinContent(2)]
        file.Close()



Welcome to JupyROOT 6.22/00


In [ ]:
#NOW ADDING THE WEIGHT-BASED SYSTEMATICS        
histograms=[]
for sys in trees[:1]:
    file=r.TFile.Open("Signal_Sherpa"+sys+".root")
    histos=file.GetListOfKeys()
    for i in histos:
        name=i.GetName()
        if ("cuts" in name) and ("tpt" in name) and ("1p" in name) and ("tau_matched" in name):
            histograms.append(i.GetName()[len("tau_matched_1p_cuts_tpt_"):])

for sam in samples:
    for sys in trees[:1]:
        file=r.TFile.Open(sam+sys+".root")
        for h in histograms:
            prong1=file.Get("tau_matched_1p_cuts_tpt_"+h)
            prong3=file.Get("tau_matched_3p_cuts_tpt_"+h)
            dict[sam+sys+h]=[prong1.GetBinContent(1),prong1.GetBinContent(2),prong3.GetBinContent(1),prong3.GetBinContent(2)]
        prong1=file.Get("tau_matched_1p_cuts_tpt_"+h)
        prong3=file.Get("tau_matched_3p_cuts_tpt_"+h)
        dict[sam+sys+"nominal"]=[prong1.GetBinContent(1),prong1.GetBinContent(2),prong3.GetBinContent(1),prong3.GetBinContent(2)]
        file.Close()

In [3]:
# REARRANGING THE DICTIONARY AS FAKES AND TRUTH EVENTS
for i in dict:
    if "Signal" not in i:
        dict[i]=[dict[i][0]+dict[i][1],0,dict[i][2]+dict[i][3],0]


In [4]:
#The structure of the dictionary is... "name_of_sys":[SherpaFake,SherpaTruth,PoPyFake,PoPyTruth,Zmumu,VV]
final_dict_1p={}
for i in histograms+trees[1:]:
    final_dict_1p[i]=[0,0,0,0,0,0]
    for j in dict:
        if (i in j) and ("Sherpa" in j):
            final_dict_1p[i][0]=dict[j][0]
            final_dict_1p[i][1]=dict[j][1]
        if (i in j) and ("PoPy" in j):
            final_dict_1p[i][2]=dict[j][0]
            final_dict_1p[i][3]=dict[j][1]
        if (i in j) and ("Zjets" in j):
            final_dict_1p[i][4]=dict[j][0]
        if (i in j) and ("VV" in j):
            final_dict_1p[i][5]=dict[j][0]
            
final_dict_3p={}
for i in histograms+trees[1:]:
    final_dict_3p[i]=[0,0,0,0,0,0]
    for j in dict:
        if (i in j) and ("Sherpa" in j):
            final_dict_3p[i][0]=dict[j][2]
            final_dict_3p[i][1]=dict[j][3]
        if (i in j) and ("PoPy" in j):
            final_dict_3p[i][2]=dict[j][2]
            final_dict_3p[i][3]=dict[j][3]
        if (i in j) and ("Zjets" in j):
            final_dict_3p[i][4]=dict[j][2]
        if (i in j) and ("VV" in j):
            final_dict_3p[i][5]=dict[j][2]

#TAKING DOWN AND UP SYSTEMATICS TO THE BOTTOM AND THE TOP OF THE DICTIONARY RESPECTIVELY
down_1p={}
down_3p={}

a1p_keys=[i for i in final_dict_1p]
a3p_keys=[i for i in final_dict_3p]

for i in a1p_keys:
    if "1down" in i:
        down_1p.update({i:final_dict_1p[i]})
        final_dict_1p.pop(i)
for i in a3p_keys:
    if "1down" in i:
        down_3p.update({i:final_dict_3p[i]})
        final_dict_3p.pop(i)
final_dict_1p.update(down_1p)
final_dict_3p.update(down_3p)

In [5]:
# EXPORTING THE RESULTS TO A CSV FILE
import pandas as pd
df1=pd.DataFrame.from_dict(final_dict_1p, orient='index',
                       columns=["SherpaFake","SherpaTruth","PoPyFake","PoPyTruth","Zjets","VV"])
df3=pd.DataFrame.from_dict(final_dict_3p, orient='index',

                       columns=["SherpaFake","SherpaTruth","PoPyFake","PoPyTruth","Zjets","VV"])

df1.to_csv('Weight-Based-Uncertainties-1prong.csv')
df3.to_csv('Weight-Based-Uncertainties-3prong.csv')


In [23]:
df1

,SherpaFake,SherpaTruth,PoPyFake,PoPyTruth,Zjets,VV
elec_0_EL_EFF_ID_TOTAL_1NPCOR_PLUS_UNCOR_1up_EleEffSF_offline_TightLLH_d0z0_v13,6.813282,7480.095215,8.399933,8054.287109,102.911743,130.041627
elec_0_EL_EFF_Iso_TOTAL_1NPCOR_PLUS_UNCOR_1up_EleEffSF_Isolation_TightLLH_d0z0_v13_FCLoose,6.731624,7414.479980,8.324745,7982.493652,102.013863,128.962977
elec_0_EL_EFF_Iso_TOTAL_1NPCOR_PLUS_UNCOR_1up_EleEffSF_Isolation_TightLLH_d0z0_v13_FCTight,6.737211,7419.854004,8.332464,7988.384766,102.092026,129.053045
elec_0_EL_EFF_Reco_TOTAL_1NPCOR_PLUS_UNCOR_1up_EleEffSF_offline_RecoTrk,6.742279,7422.982422,8.335152,7991.686035,102.122231,129.101890
elec_0_EL_EFF_Trigger_TOTAL_1NPCOR_PLUS_UNCOR_1up_EleEffSF_SINGLE_E_2015_e24_lhmedium_L1EM20VH_OR_e60_lhmedium_OR_e120_lhloose_2016_2018_e26_lhtight_nod0_ivarloose_OR_e60_lhmedium_nod0_OR_e140_lhloose_nod0_TightLLH_d0z0_v13_isolFCTight,6.738810,7428.098633,8.340236,7997.749512,102.237473,129.175465
...,...,...,...,...,...,...
TAUS_TRUEHADTAU_SME_TES_DETECTOR_1down,6.729600,7546.510742,8.322298,8124.981445,101.988487,132.041356
TAUS_TRUEHADTAU_SME_TES_INSITUEXP_1down,6.729600,7341.930176,8.322298,7899.375000,101.988487,128.149053
TAUS_TRUEHADTAU_SME_TES_INSITUFIT_1down,6.729600,7297.691406,8.322298,7822.201172,101.988487,127.257688
TAUS_TRUEHADTAU_SME_TES_MODEL_CLOSURE_1down,6.729600,7403.116699,8.322298,7962.165039,101.988487,128.761586
